## Basic code using spaudiopy package
- Docs: https://spaudiopy.readthedocs.io/en/latest/index.html
- GitHub: https://github.com/chris-hld/spaudiopy

In [ ]:
# Check spaudiopy.sig functions to open MONO signal and play with HRIRs
import spaudiopy as spa
from spaudiopy.sig import MonoSignal as ms
from spaudiopy.sig import MultiSignal as stereo 
import spaudiopy.process as sproc

import scipy.signal
from scipy.io import wavfile
from IPython.display import Audio

fs=44100 # sampling rate
import numpy as np

## Tools
We use three parameters to locate an HRTF:
1. *Azimuth*: angle between position and sound location on the $xy$-plane
2. *Elevation*: angle between position and sound location on the $xz$-plane
3. *Time* or *Frequency*: time period or frequency of emitted sound w.r.t. actual position

Math tools:
- *Haversine distance*: Consider two points $x_1$ and $x_2$ on a sphere with respective latitudes and longitudes $(\varphi_1,\varphi_2)$ and $(\theta_1,\theta_2)$. The Haversine distance $D(x_1,x_2)$ is the angular distance between them on the surface of the sphere given by $$D(x_1,x_2)=2\arcsin\sqrt{\sin^2\left(\frac{\varphi_2-\varphi_1}{2}\right)+\cos x_1\cos y_1\sin^2\left(\frac{\theta_2-\theta_1}{2}\right)}.$$ We use this distance when wanting to find the closest HRIR point from a grid.

In [ ]:
piano_test = ms.from_file("piano_testfile.wav")
piano_test.trim(0,3)
#piano_test.play()

## Test delay on stereo file

In [ ]:
# Initial test
stereo_sample = stereo.from_file("ocean_eyes.wav")
s1, s2 = stereo_sample.get_signals()

In [ ]:
delay = np.zeros(len(stereo_sample), dtype=int)
delay[int(fs*0.03)] = 1
delayed_s2 = scipy.signal.convolve(s2, delay)[:len(stereo_sample)]
delayed_stereo = stereo([s1, delayed_s2], fs=fs)

In [ ]:
display(Audio(stereo_sample, rate=fs)) # original sample
#display(Audio(delayed_stereo, rate=fs))

In [ ]:
def get_stereo_tracks(music_file):
    sample = stereo.from_file(music_file)
    return sample.get_signals()

In [ ]:
MS_DELAY = 25 # delay in [ms] to create surrounding effect

def delay_signal(s1, s2):
    assert(len(s1) == len(s2))
    _len_sample = len(s1)
    delay = np.zeros(_len_sample, dtype=int)
    delay[int(fs*MS_DELAY / 1000)] = 1
    return scipy.signal.convolve(s2, delay)[:_len_sample]

def spatialise_sound(music_file, left=False):
    s1,s2 = get_stereo_tracks(music_file)
    if left:
        delayed_s1 = delay_signal(s2, s1)
        return stereo([delayed_s1, s2], fs=fs)
    delayed_s2 = delay_signal(s1, s2)
    return stereo([s1, delayed_s2], fs=fs)

s_left = spatialise_sound("ocean_eyes.wav", True)
display(Audio(s_left, rate=fs))
#s_right = spatialise_sound("ocean_eyes.wav", False)
#display(Audio(s_right, rate=fs))

## Use of localisation cues

In [ ]:
avg_ear_distance = 0.21 # 21cm
speed_of_sound = 343 # in m/s

def fusion(s1, s2):
    assert(len(s1) == len(s2))
    s = []
    for i in range(len(s1)):
        s.append(s1[i]/2 + s2[i]/2)
    return s

def compute_delay_from_angle(rad):
    return avg_ear_distance * np.sin(rad) / speed_of_sound

def attenuate(sig, direction):
    return [e*np.cos(direction/4) for e in sig]

def localise_sound(stereo_music_file, direction, left=True):
    """Change location of the sound source
        @param stereo_music_file: music file we wish to use
        @param direction: where we want the sound to come from
        @param left: False if we want the sound to come from the right, True o/wise
    """
    s1,s2 = get_stereo_tracks(music_file)
    
    sample_song_mono = fusion(s1, s2)
    maxmono = max(sample_song_mono)
    mono_norm = []
    for i in range(len(sample_song_mono)):
        mono_norm.append(sample_song_mono[i]/(maxmono+1))
        
    delay = np.zeros(len(mono_norm), dtype=int)
    delay[int(fs*compute_delay_from_angle(direction))] = 1
    delayed = scipy.signal.convolve(mono_norm, delay)[:len(mono_norm)]
    
    if left:
        return stereo([mono_norm, attenuate(delayed, direction)], fs=fs) # sound to the left
    return stereo([attenuate(delayed, direction), mono_norm], fs=fs) # sound to the right

In [ ]:
s_west = localise_sound("ocean_eyes.wav", np.pi/2)
display(Audio(s_ww, rate=fs))